In [ ]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict

def q_learning_fixed_with_default_action(data, alpha=0.1, gamma=1, epsilon=0.1, default_action='No Training'):
    """
    Perform Q-learning on the given dataset and calculate the policy,
    defaulting to a specific action when no actions are available in the Q-table.
    """
    q_table = defaultdict(lambda: defaultdict(float))

    for epoch in range(500):
        if epoch % 10 == 0:
            print(epoch)
        for i in range(len(data)):
            sample_row = data.iloc[i]
            state = (sample_row['Job Title Numeric'], sample_row['Years of Experience in Months'], 
                    sample_row['P_Score'], sample_row['C_Rating'])
            action = sample_row['Training Program']
            reward = sample_row['Reward']
            next_state = (sample_row['Job Title Numeric'], sample_row['Next Years of Experience in Months'], 
                        sample_row['Next P_Score'], sample_row['Next C_Rating'])

            old_value = q_table[state][action]
            next_max = np.max(list(q_table[next_state].values())) if q_table[next_state] else 0
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[state][action] = new_value

    optimal_policy = {}
    for state in q_table:
        if q_table[state]:
            best_action = max(q_table[state], key=q_table[state].get)
            optimal_policy[state] = best_action
        else:
            optimal_policy[state] = default_action

    return q_table, optimal_policy

def sarsa_with_default_action(data, alpha=0.1, gamma=1, epsilon=0.1, default_action='No Training'):
    """
    Perform SARSA on the given dataset and calculate the policy,
    defaulting to a specific action when no actions are available in the Q-table.
    """
    q_table = defaultdict(lambda: defaultdict(float))

    for epoch in range(500):
        if epoch % 10 == 0:
            print(epoch)
        
        for i in range(len(data) - 1):  # Loop over data, leaving the last row for terminal state
            sample_row = data.iloc[i]
            next_row = data.iloc[i + 1]

            # Current state and action
            state = (sample_row['Job Title Numeric'], sample_row['Years of Experience in Months'], 
                     sample_row['P_Score'], sample_row['C_Rating'])
            action = sample_row['Training Program']

            # Next state and action (using next row in the data)
            next_state = (next_row['Job Title Numeric'], next_row['Years of Experience in Months'], 
                          next_row['Next P_Score'], next_row['Next C_Rating'])
            next_action = next_row['Training Program'] if (i < len(data) - 1) else default_action

            # SARSA update rule
            reward = sample_row['Reward']
            old_value = q_table[state][action]
            next_value = q_table[next_state][next_action]
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_value)
            q_table[state][action] = new_value

    # Determine the optimal policy from the Q-table
    optimal_policy = {}
    for state in q_table:
        if q_table[state]:
            best_action = max(q_table[state], key=q_table[state].get)
            optimal_policy[state] = best_action
        else:
            optimal_policy[state] = default_action

    return q_table, optimal_policy

# Note: This function assumes the dataset is ordered such that consecutive rows represent
# consecutive steps in an episode. If this is not the case, additional logic will be needed
# to correctly pair states and actions.



def calculate_average_policy_value(q_table, initial_states, gamma=0.9, max_steps=100):
    """
    Calculate the average value of the policy over a set of initial states, using the Q-table.
    """
    total_values = []

    for initial_state in initial_states:
        total_reward = 0
        current_state = initial_state
        for step in range(max_steps):
            best_action = None
            best_q_value = float('-inf')
            for action, q_value in q_table[current_state].items():
                if q_value > best_q_value:
                    best_q_value = q_value
                    best_action = action

            if best_action is None:
                break

            total_reward += (gamma ** step) * best_q_value
            current_state = tuple(x + 1 for x in current_state)
        total_values.append(total_reward)

    return sum(total_values) / len(total_values) if total_values else 0

def calculate_random_policy_value(data, initial_states, gamma=0.9, max_steps=100):
    """
    Calculate the average value of a random policy over a set of initial states.
    """
    total_values = []

    for initial_state in initial_states:
        total_reward = 0
        current_state = initial_state
        for step in range(max_steps):
            available_actions = data[data['Job Title Numeric'] == current_state[0]]['Training Program'].unique()
            if not len(available_actions):
                break

            action = random.choice(available_actions)
            sample_row = data[(data['Job Title Numeric'] == current_state[0]) & 
                            (data['Training Program'] == action)].sample(n=1)
            reward = sample_row['Reward'].iloc[0]

            total_reward += (gamma ** step) * reward
            current_state = tuple(x + 1 for x in current_state)
        total_values.append(total_reward)

    return sum(total_values) / len(total_values) if total_values else 0

# First, I will load and inspect the contents of the provided dataset to understand its structure and the data available.
import pandas as pd
import random

# Load the dataset
file_path = '/Users/aditribhagirath/Desktop/mediumjob.csv'
data = pd.read_csv(file_path, delimiter='\t')
print(len(data))


# Display the first few rows of the dataset to understand its structure
#print(data.head())

from collections import defaultdict
import numpy as np


q_table, optimal_policy = sarsa_with_default_action(data)
sample_initial_states = list(q_table.keys())
learned_policy_value = calculate_average_policy_value(q_table, sample_initial_states)
random_policy_value = calculate_random_policy_value(data, sample_initial_states)
print(learned_policy_value, random_policy_value)
